# Lakh MIDI DataSet

Things to consider

- Multi-threaded encoding
- Filter out drums (track 10)

In [1]:
from pathlib import Path
import music21 as m21
import random
import pretty_midi
import IPython.display
musescore_path = '/usr/bin/mscore'
m21.environment.set('musicxmlPath', musescore_path)
m21.environment.set('musescoreDirectPNGPath', musescore_path)

In [2]:
midi_path = Path('data/midi/lakh_clean')
midi_file_paths = list(midi_path.rglob('*.mid'))
len(midi_file_paths)

17256

In [ ]:
midi_file_paths[0:5]

In [ ]:
file_path = "data/midi/lakh_clean/Cash Johnny/A Boy Named Sue.mid" #/Technotronic/Pump Up the Jam.1.mid"# Cash Johnny/A Boy Named Sue.mid" # midi_file_paths[random.randint(0, len(midi_file_paths))]
print(file_path)
midi_file = m21.midi.MidiFile()
midi_file.open(file_path)
midi_file.read()
midi_file.close()

midi_file

The Music21 renderer can't play multi track files :(

In [ ]:
midi_stream = m21.midi.translate.midiFileToStream(midi_file)
midi_stream.show('midi')

But PrettyMidi + FluidSynth + IPython.display can!

In [ ]:
pretty_midi_data = pretty_midi.PrettyMIDI(str(file_path))

audio_data = pretty_midi_data.fluidsynth()
IPython.display.Audio(audio_data, rate=44100)

Music21 does actually have the data, it is just the audio rendering in Jupyter that is glitched

In [ ]:
midi_stream.show()

In [ ]:
pretty_midi_data.instruments = [i for i in pretty_midi_data.instruments if not i.is_drum]

for i in pretty_midi_data.instruments:
    i.program = 0

audio_data = pretty_midi_data.fluidsynth()
IPython.display.Audio(audio_data, rate=44100)

In [ ]:
random_path = midi_file_paths[random.randint(0, len(midi_file_paths))]
print(random_path)
random_pretty_data = pretty_midi.PrettyMIDI(str(random_path))

random_audio_data = random_pretty_data.fluidsynth()
IPython.display.Audio(random_audio_data, rate=44100)

In [ ]:
random_pretty_data.instruments = [i for i in random_pretty_data.instruments if not i.is_drum]
for i in random_pretty_data.instruments:
    i.program = 0

piano_audio_data = random_pretty_data.fluidsynth()
IPython.display.Audio(piano_audio_data, rate=44100)

In [ ]:
from multiprocessing import Pool

midi_piano_path = Path('data/midi/lakh_clean_piano')
midi_piano_path.mkdir(exist_ok=True)

def process_midi_file(midi_file_path):
    try:
        pretty_data = pretty_midi.PrettyMIDI(str(midi_file_path))
        pretty_data.instruments = [i for i in pretty_data.instruments if not i.is_drum]
        for i in pretty_data.instruments:
            i.program = 0
        new_path = Path(midi_piano_path, midi_file_path.parts[-2])
        new_path.mkdir(exist_ok=True)
        new_file_path = Path(new_path, midi_file_path.parts[-1])
        if not new_file_path.exists():
            pretty_data.write(str(new_file_path))
            print(f'Wrote {str(new_file_path)}')
        else:
            print(f'Skipped {str(new_file_path)}')
    except Exception as e:
        print(f'Error: {e}')

if __name__ == '__main__':
    with Pool(processes=24) as pool:  # Adjust the number of processes based on your system
        pool.map(process_midi_file, midi_file_paths)